In [29]:
:opt no-lint

In [1]:
data Expr = Val Int | Add Expr Expr  deriving Show

Step 1. Define the Semantics

In [2]:
eval :: Expr -> Int
eval (Val n)   = n
eval (Add x y) = eval x + eval y

In [3]:
eval (Add (Val 3) (Val 4))

7

Step 2. 두번째 변형된 의미 stack transformer (사실상 가상머신 정의에 가까움)

만족하기 원하는 성질
```
evalS e s = eval e : s
```

In [10]:
type Stack = [Int]

In [15]:
pushS :: Int -> Stack -> Stack
pushS n s = n : s 

addS :: Stack -> Stack
addS (n:m:s) = (m+n):s

In [16]:
evalS :: Expr -> Stack -> Stack
evalS (Val n)   s = pushS n s
evalS (Add x y) s = addS (evalS y (evalS x s))

Step 3. CPS style

흐름 제어(즉, 계산의 순서)를 명확이 드러내기 위해

(((((      (1 + 2) + (3 + 4)      ))))))))

만족하고자 하는 성질
```
evalC e c s = c (evalS e s)
```

In [17]:
type Cont = Stack -> Stack

In [20]:
evalC :: Expr -> Cont -> Cont
evalC (Val n) c s   = c (pushS n s)
evalC (Add x y) c s = evalC x (evalC y (c . addS)) s

```
  evalC ((1+2)+(3+4)) c s
= evalC (1+2) (evalC (3+4) (c . addS)) s
= evalC 1 (evalC 2 ((evalC (3+4) (c . addS)) . addS)) s
= (evalC 2 ((evalC (3+4) (c . addS)) . addS)) (pushS 1 s)
= (((evalC (3+4) (c . addS)) . addS) (pushS 2 s)) (pushS 1 s)
```

Step 4. 좀더 깔끔하게 CPS 정리

In [30]:
type Cont = Stack -> Stack

pushC :: Int -> Cont -> Cont
pushC n c = c . pushS n

addC :: Cont -> Cont
addC c = c . addS

haltC :: Cont    -- 더 이상 나머지 할 일(포함하고 있는 더 큰 식)이 없다
haltC = \s -> s

In [31]:
evalC :: Expr -> Cont -> Cont
evalC (Val n) c   = pushC n c
evalC (Add x y) c = evalC x (evalC y (addC c))

계산과정에서 기본 연산(addC, pushC)에 해당하는 개념을 데이터화하면

구분해서 검사 가능 (함수 안은 일반적으로 들여다보고 검사가 안되니까)

In [39]:
data Code
  = HALT
  | PUSH Int Code
  | ADD Code
  deriving Show

In [35]:
:type HALT
:type PUSH
:type ADD

HALT :: Code

PUSH :: Int -> Code -> Code

ADD :: Code -> Code -> Code

In [43]:
-- 이거 유도과정부터 다시 시작하면 될 듯
exec :: Code -> Stack -> Stack
exec HALT          s = s
exec (PUSH n c)    s = exec c (n : s) 
exec (ADD c) (n:m:s) = exec c ((m+n):s)

In [ ]:
Expr -> Code